In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

column_names = ['title', 'col1', 'col2', 'col3', 'col4', 'col5', 'col6']
data = pd.read_csv('/content/drive/My Drive/twitter/twitter_sample_tweets.csv', names=column_names, error_bad_lines=False, nrows=10000, encoding='utf-8')


Mounted at /content/drive


<ipython-input-2-c9617d414cdc>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv('/content/drive/My Drive/twitter/twitter_sample_tweets.csv', names=column_names, error_bad_lines=False, nrows=10000, encoding='utf-8')


In [ ]:
import re

def preprocess_text(text):
    # Convert text to lowercase
    # text = text.lower()

    # Remove URLs and mentions
    text = re.sub(r"http\S+", "", str(text))
    text = re.sub(r"@\S+", "", (text))
    text = re.sub(r'#\w+', '', (text))

    return text

# Preprocess the data
data['title'] = data['title'].apply(preprocess_text)

In [ ]:
data.head()

,title,col1,col2,col3,col4,col5,col6
2019-02-01 00:00:00,RT : به نظر شما وقتی وزیر امورخارجه بریتانیا ...,981,0,0,144,220,184
2019-02-01 00:00:00,RT : بنر تصاویر دیکتاتور توسط به آتش کشیده شد.,163,0,1,1589,981,31282
2019-02-01 00:00:00,RT : مشهد به واسطه وجود حرم مطهر امام رضا(ع) و...,60,0,2,380,418,3614
2019-02-01 00:00:00,عاخه تو مغزشون کردن هرکسی که مثلا از نظام انت...,0,1,3,466,708,5421
2019-02-01 00:00:00,نرم باشه لطفاً.,0,2,4,381,755,4084


In [ ]:
# Split the data into training, validation, and test datasets
train_size = int(0.7 * len(data))
val_size = int(0.15 * len(data))
test_size = len(data) - train_size - val_size

train_data = data[:train_size]
val_data = data[train_size:train_size+val_size]
test_data = data[train_size+val_size:]

In [ ]:
# Tokenize the text
tokenizer = Tokenizer(oov_token='<oov>')
tokenizer.fit_on_texts(train_data['title'])

In [ ]:
# Convert the text to sequences of integers
train_sequences = tokenizer.texts_to_sequences(train_data['title'])
val_sequences = tokenizer.texts_to_sequences(val_data['title'])
test_sequences = tokenizer.texts_to_sequences(test_data['title'])

In [ ]:
# Pad the sequences
max_sequence_len = max([len(seq) for seq in train_sequences])
train_sequences = pad_sequences(train_sequences, maxlen=max_sequence_len, padding='pre')
val_sequences = pad_sequences(val_sequences, maxlen=max_sequence_len, padding='pre')
test_sequences = pad_sequences(test_sequences, maxlen=max_sequence_len, padding='pre')

In [ ]:
# Convert the labels to one-hot encoding
num_words = len(tokenizer.word_index) + 1
train_labels = to_categorical(train_data['col1'], num_classes=num_words)
val_labels = to_categorical(val_data['col1'], num_classes=num_words)
test_labels = to_categorical(test_data['col1'], num_classes=num_words)

In [ ]:
# Define the model
model = Sequential()
model.add(Embedding(num_words, 100, input_length=max_sequence_len))
model.add(Bidirectional(LSTM(150, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(num_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(train_sequences, train_labels, epochs=2, batch_size=32, validation_data=(val_sequences, val_labels))


Epoch 1/2
219/219 [==============================] - 172s 755ms/step - loss: 3.3423 - accuracy: 0.5451 - val_loss: 2.6846 - val_accuracy: 0.5560
Epoch 2/2
219/219 [==============================] - 155s 708ms/step - loss: 2.4038 - accuracy: 0.5480 - val_loss: 2.5663 - val_accuracy: 0.5547


In [ ]:
# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_sequences, test_labels)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

47/47 [==============================] - 6s 131ms/step - loss: 2.6012 - accuracy: 0.5427
Test loss: 2.601213216781616
Test accuracy: 0.5426666736602783


In [ ]:
index_to_word = {index: word for word, index in tokenizer.word_index.items()}
import numpy as np
import pickle

max_sequence_len = 64


In [ ]:
import numpy as np

# Preprocess the input text
input_text = "ایران"
input_text = preprocess_text(input_text)
input_tokens = tokenizer.texts_to_sequences([input_text])
input_sequence = pad_sequences(input_tokens, maxlen=max_sequence_len-1, padding='pre')

# Use the trained model to make predictions
predictions = model.predict(input_sequence)[0]

# Get the top 5 predicted indices and their probabilities
top_indices = predictions.argsort()[-5:][::-1]
top_probabilities = predictions[top_indices]

# Normalize the probabilities
norm_probabilities = top_probabilities / np.sum(top_probabilities)

# Choose a predicted index at random based on the probabilities
predicted_index = np.random.choice(top_indices, p=norm_probabilities)

# Convert the predicted index to a word
if predicted_index in index_to_word:
    predicted_word = index_to_word[predicted_index]
    print(predicted_word)
else:
    print("Predicted index not in index_to_word dictionary")

1/1 [==============================] - 0s 47ms/step
به
